# **Using multivariate polynomial regression to predict joint movement of robotic arm to improve the accuracy of movement simulations.**

## Problem statement

## imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

## Datasets

In [ ]:
sim_df